In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv

In [ ]:
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.info()
# the data appears to be ok, just encode the strings collumns

In [ ]:
# encode sex as 0 male and 1 female
dataset['sex'] = np.where(dataset['sex'] == 'male', 0, 1)
dataset['sex'].value_counts()
# values seen balanced

In [ ]:
# encode sex as 0 male and 1 female
dataset['smoker'] = np.where(dataset['smoker'] == 'yes', 1, 0)
dataset['smoker'].value_counts()
# values seen balanced

In [ ]:
dataset['region'].value_counts()

In [ ]:
# Do one-hot encoding for region (why pandas call dummies?)
one_hot_encoding_regions = pd.get_dummies(dataset['region'], dtype=np.uint8, drop_first=True)
dataset = pd.concat([dataset, one_hot_encoding_regions], axis=1)
dataset.head()

In [ ]:
dataset = dataset.drop(columns=['region'])
dataset.info()

In [ ]:
import seaborn as sns
plt.figure(figsize=(16,10))
sns.heatmap(dataset.corr(), annot=True)
plt.show()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
x = dataset.drop(columns=['expenses'])
y = dataset['expenses']

# as asked: Use 80% of the data as the train_dataset and 20% of the data as the test_dataset.
train_dataset, test_dataset, train_labels, test_labels = train_test_split(x, y, test_size=0.2, random_state=49)
pipe = make_pipeline(StandardScaler(), LinearRegression())
pipe.fit(train_dataset, train_labels)  # apply scaling on training data

In [ ]:
train_dataset.shape, test_dataset.shape, train_labels.shape, test_labels.shape

## I only now realized that the test will only take a keras model

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
normilized_train_dataset = scaler.fit_transform(train_dataset)
normilized_train_labels = scaler.transform(test_dataset)

In [ ]:
normilized_train_dataset.shape, normilized_test_dataset.shape # got in the way more than helped

In [ ]:
model = keras.Sequential()

model.add(layers.Dense(64, input_dim=train_dataset.shape[1], activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1))

model.compile(loss='mse', # mean squared error
              optimizer='adam',
              metrics=['mae', 'mse']) # mae: mean absolute error

model.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

history = model.fit(train_dataset,
                    train_labels,
                    epochs=400,
                    batch_size=32,
                    validation_split = 0.1,
                    verbose=0,
                    callbacks=[early_stop, PrintDot()])

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.legend()
  plt.show()


plot_history(history)

In [ ]:
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
